In [18]:
import math
import numpy as np
import sys
from typing import List
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
from importlib import reload
reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ls
import graphs as g
from lists import Node
print()

In [246]:
def search(lt, l, h, d):
    while l <= h:
        m = (l + h) // 2
        if lt(m):
            l = m + 1
        else:
            d = m
            h = m - 1
    return d

def min_sum_split_array(a: list, k: int) -> int:
    "Split `a` in `k` sub-sequences. Minimize the maximum sum of every one."
    acc = lambda mx: lambda a, e: (e, a[1] + 1) if a[0] + e > mx else (a[0] + e, a[1])
    fit = lambda mx: k < reduce(acc(mx), a, (0, 1))[1]
    return search(fit, max(a), sum(a), -1)

In [248]:
min_sum_split_array([1, 2, 3, 4], 1)

10